In [4]:
!apt-get install -y \
    r-base \
    r-base-dev \
    cmake \
    libnlopt-dev 

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package r-base
E: Unable to locate package r-base-dev
E: Unable to locate package cmake
E: Unable to locate package libnlopt-dev


R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In (function (pkgs, lib, repos = getOption("repos"), contriburl = contrib.url(repos,  :
R[write to console]: 
 
R[write to console]:  installation of package ‘nloptr’ had non-zero exit status

R[write to console]: 2: 
R[write to console]: In (function (pkgs, lib, repos = getOption("repos"), contriburl = contrib.url(repos,  :
R[write to console]: 
 
R[write to console]:  installation of package ‘lme4’ had non-zero exit status



In [1]:
!pip install statsmodels patsy pymer4 rpy2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.9/136.9 kB 15.0 MB/s eta 0:00:00


In [2]:
# Import rpy2
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

# Install the utils package to manage installation
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)  # Select the first mirror in the list

# Define the packages to be installed
packnames = ('nloptr', 'lme4')

# Check which packages are not installed
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]

# Install the missing packages
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))


ℹ Using R 4.3.1 (lockfile was generated with R 4.4.1)
- Project '/local' loaded. [renv 1.0.7]
- One or more packages recorded in the lockfile are not installed.
- Use `renv::status()` for more details.


R[write to console]: Installing packages into ‘/local/renv/library/R-4.3/x86_64-conda-linux-gnu’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/nloptr_2.1.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2236635 bytes (2.1 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

checking whether the C++ compiler works... yes
checking for C++ compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether the compiler supports GNU C++... yes
checking whether x86_64-conda-linux-gnu-c++ -std=gnu++17 accepts -g... yes
checking for x86_64-conda-linux-gnu-c++ -std=gnu++17 option to enable C++11 features... none needed
checking how to run the C++ preprocessor... x86_64-conda-linux-gnu-c++ -std=gnu++17 -E
checking whether the compiler supports GNU C++... (cached) yes
checking whether x86_64-conda-linux-gnu-c++ -std=gnu++17 accepts -g... (cached) yes
checking for x86_64-conda-linux-gnu-c++ -std=gnu++17 option to enable C++11 features... (cached) none needed
checking for pkg-config... no
checking for cmake... no

------------------ CMAKE NOT FOUND --------------------

CMake was not found on the PATH. Please install CMake:

 - sudo yum install cmake 

ERROR: configuration failed for package ‘nloptr’
* removing ‘/local/renv/library/R-4.3/x86_64-conda-linux-gnu/nloptr’
ERROR: dependency ‘nloptr’ is not available for package ‘lme4’
* removing ‘/local/renv/library/R-4.3/x86_64-conda-linux-gnu/lme4’
R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/tmp/Rtmpw1Vw8n/downloaded_packages’
R[write to console]: 
R[write to console]: 



In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod.bayes_mixed_glm import PoissonBayesMixedGLM
from tqdm import tqdm
from patsy import dmatrices
from patsy import dmatrix
from statsmodels.discrete.count_model import ZeroInflatedPoisson

In [ ]:
object_data = pd.read_csv('./2024_Jul_ob_count.csv')

In [ ]:
# Create array of category names as they appear in the detections data. See paper for details of each category.
categories = ['car', 'person', 'trotro', 'stall', 'truck', 'stove', 'motorcycle', 'vendor', 'lorry', 'umbrella', 'bus', 'trash', 'taxi', 'van', 'debris', 'loudspeaker', 'bowl', 'food', 'animal', 'bicycle']

# Column names in the data frame for the number of counts of each category type in an image.
count_cols = [cat+'_counts' for cat in categories]

super_count_cols = ['people'+'_counts', 'small_vehicles'+'_counts', 'two_wheelers'+'_counts', 'large_vehicles'+'_counts', 'refuse'+'_counts', 'market'+'_counts', 'animal'+'_counts']

all_count_cols = count_cols + super_count_cols

vehicle_categories = ['car', 'trotro', 'truck', 'motorcycle', 'lorry', 'bus', 'taxi', 'van', 'bicycle']

# Define super categories
super_categories = {
    'people': ['person', 'vendor'],
    'small_vehicles': ['car', 'taxi', 'truck'],
    'two_wheelers': ['bicycle', 'motorcycle'],
    'large_vehicles': ['trotro', 'van', 'lorry', 'bus'],
    'refuse': ['trash', 'debris'],
    'market': ['umbrella', 'stall', 'bowl', 'food'],
    'animal': ['animal']
}

In [ ]:
# Ensure datetime is in datetime format
object_data['datetime'] = pd.to_datetime(object_data['datetime_rectified'], format='%Y-%m-%d %H:%M:%S')

# Create additional time-related columns
object_data['hour'] = object_data['datetime'].dt.hour
object_data['day'] = object_data['datetime'].dt.dayofweek + 1  # +1 to match R's 1-indexing
object_data['week'] = object_data['datetime'].dt.isocalendar().week
object_data['year'] = object_data['datetime'].dt.year

# Split 'site_id_cam_angle' into 'site_id' and 'camera' columns
object_data[['site_id', 'camera']] = object_data['site_id_cam_angle'].str.split('_', expand=True)

# Fill missing values in 'camera' with 'single'
object_data['camera'].fillna('single', inplace=True)

# Filter data between specified dates
start_date = pd.Timestamp('2019-04-01')
end_date = pd.Timestamp('2024-04-01')
fixed_object_data = object_data[(object_data['datetime'] >= start_date) & (object_data['datetime'] <= end_date)]
fixed_object_data = fixed_object_data[fixed_object_data['view'] == 'clear']
# Display the first few rows of the new dataframe to verify
fixed_object_data.head()


In [ ]:
# Sum counts for each super category
for super_cat, categories in super_categories.items():
    # Create a column for each supercategory by summing its categories
    fixed_object_data[super_cat + '_counts'] = fixed_object_data[[cat + '_counts' for cat in categories]].sum(axis=1)

In [ ]:
# Step 7: Round the 'datetime' to the nearest hour
fixed_object_data['datetime_hour'] = fixed_object_data['datetime'].dt.round('H')

# Step 8: Sum the counts within each hour for each camera at each site
hourly_counts = fixed_object_data.groupby(['site_id', 'camera', 'datetime_hour'])[all_count_cols].sum().reset_index()

# Step 9: Group by the rounded 'datetime', 'date', and 'site_id', then calculate the mean for each object category
hourly_averages = hourly_counts.groupby(['site_id', 'datetime_hour'])[all_count_cols].mean().reset_index()

# Step 10: Add the date column from the rounded 'datetime_hour'
hourly_averages['date'] = hourly_averages['datetime_hour'].dt.date

# Create additional time-related columns
hourly_averages['hour'] = hourly_averages['datetime_hour'].dt.hour
hourly_averages['day'] = hourly_averages['datetime_hour'].dt.dayofweek + 1  # +1 to match R's 1-indexing
hourly_averages['week'] = hourly_averages['datetime_hour'].dt.isocalendar().week
hourly_averages['year'] = hourly_averages['datetime_hour'].dt.year


# Step 11: Reorder the columns to match the requested format
final_columns = ['datetime_hour', 'date', 'site_id', 'hour', 'day', 'week', 'year'] + all_count_cols
hourly_averages = hourly_averages[final_columns]

# Step 12: Rename 'datetime_hour' to 'datetime' to match the final requested column name
hourly_averages.rename(columns={'datetime_hour': 'datetime'}, inplace=True)

# Display the first few rows of the new dataframe to verify
hourly_averages.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
from pymer4.models import Lmer

# Step 1: Adjust the 'year' column to start from 0
hourly_averages['year'] = hourly_averages['year'] - hourly_averages['year'].min()

# Step 2: Filter the data to include only those rows with counts > 0
filtered_data = hourly_averages[hourly_averages['people_counts'] > 0]

# Step 3: One-hot encode 'hour', 'day', 'week', and 'site_id' for fixed effects
exog_fixed = pd.get_dummies(filtered_data[['hour', 'day', 'week', 'site_id']], drop_first=True)

# Step 4: Ensure that 'year' remains as integer and add it to exog_fixed
exog_fixed['year'] = filtered_data['year']

# Step 5: Standardize only the 'year' column
# scaler = StandardScaler()
# exog_fixed[['year']] = scaler.fit_transform(exog_fixed[['year']])

# Step 6: Prepare the dataset for pymer4
data_for_pymer = pd.concat([filtered_data[['people_counts', 'hour', 'day', 'week', 'site_id']], exog_fixed], axis=1)

# Step 7: Create the formula for the mixed model
fixed_effects_formula = ' + '.join(exog_fixed.columns)
formula = f'people_counts ~ {fixed_effects_formula} + (1|hour) + (1|day) + (1|week) + (1|site_id)'

# Step 8: Fit the Poisson mixed model
model = Lmer(formula, data=data_for_pymer, family='poisson')
result = model.fit()

# Step 9: Display the results
print(result)


In [ ]:
from statsmodels.genmod.bayes_mixed_glm import PoissonBayesMixedGLM
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Step 1: Adjust the 'year' column to start from 0
hourly_averages['year'] = hourly_averages['year'] - hourly_averages['year'].min()

# Step 2: Filter the data to include only those rows with counts > 0
filtered_data = hourly_averages[hourly_averages['people_counts'] > 0]

# Step 3: Create the endogenous variable (response variable)
endog = filtered_data['people_counts'].astype(int)

# Step 4: One-hot encode 'hour', 'day', 'week', and 'site_id' for fixed effects
exog_fixed = pd.get_dummies(filtered_data[['hour', 'day', 'week', 'site_id']], drop_first=True)

# Step 5: Ensure that 'year' remains as integer and add it to exog_fixed
exog_fixed['year'] = filtered_data['year']

# Step 6: Standardize only the 'year' column
scaler = StandardScaler()
exog_fixed[['year']] = scaler.fit_transform(exog_fixed[['year']])

# Step 7: One-hot encode 'hour', 'day', 'week', and 'site_id' for random effects
exog_re = pd.get_dummies(filtered_data[['hour', 'day', 'week', 'site_id']], drop_first=True)

# Step 8: Convert all boolean columns to integers to ensure no object types remain
exog_fixed = exog_fixed.applymap(lambda x: int(x) if isinstance(x, bool) else x)
exog_re = exog_re.applymap(lambda x: int(x) if isinstance(x, bool) else x)

# Debugging: Check the data types and head of exog_fixed and exog_re
print("Data types of exog_fixed after conversion:")
print(exog_fixed.dtypes)
print("\nexog_fixed after conversion:")
print(exog_fixed.head())

print("\nData types of exog_re after conversion:")
print(exog_re.dtypes)
print("\nexog_re after conversion:")
print(exog_re.head())

# Step 9: Specify the ident vector, indicating which columns in exog_re identify the random effects
ident = np.zeros(exog_re.shape[1], dtype=int)

# Step 10: Fit the Poisson mixed-effects model
model = PoissonBayesMixedGLM(endog, exog_fixed, exog_re, ident=ident, vcp_p=1.0)
result = model.fit_vb()

# Step 11: Display the results
print(result.summary())


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrix
import numpy as np
# Define the counts column (e.g., 'car_counts') to model
counts_column = 'person_counts'

# Ensure all necessary data types are compatible and treat them as categorical
model_fixed_object_data = hourly_averages[hourly_averages[counts_column] > 0].copy()
model_fixed_object_data['hour'] = model_fixed_object_data['hour'].astype('category')
model_fixed_object_data['day'] = model_fixed_object_data['day'].astype('category')
model_fixed_object_data['week'] = model_fixed_object_data['week'].astype('category')
model_fixed_object_data['year'] = model_fixed_object_data['year'].astype('category')
model_fixed_object_data['site_id'] = model_fixed_object_data['site_id'].astype('category')

# Fit the fixed effects model using GLM with the negative binomial family
glm_formula = f'{counts_column} ~ C(year)'
glm_model = smf.glm(formula=glm_formula, data=model_fixed_object_data, family=sm.families.NegativeBinomial())
glm_results = glm_model.fit()

# Print the summary of the fixed effects model
print(glm_results.summary())

# Add the predicted fixed effects to the dataset
model_fixed_object_data['fixed_effects'] = glm_results.fittedvalues

# Construct the formula for the mixed effects model
mixed_effects_formula = 'fixed_effects ~ 1'

# Fit the mixed effects model using MixedLM
mixed_model = smf.mixedlm(mixed_effects_formula, model_fixed_object_data, groups=model_fixed_object_data['site_id'], re_formula="~C(hour)+C(day)+C(week)")
mixed_results = mixed_model.fit()

# Print the summary of the mixed effects model
print(mixed_results.summary())